In [1]:
import tableauserverclient as TSC
from tableauserverclient.models import WorkbookItem
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
e = load_dotenv(".tableau_env")

In [3]:
TABLEAU_USERNAME = os.environ.get("TABLEAU_USERNAME")
TABLEAU_PASSWORD = os.environ.get("TABLEAU_PASSWORD")
TABLEAU_SITENAME = os.environ.get("TABLEAU_SITENAME")
TABLEAU_SERVER_URL = os.environ.get("TABLEAU_SERVER_URL")
# for auth via token
TABLEAU_TOKEN_NAME = os.environ.get("TABLEAU_TOKEN_NAME")
TABLEAU_TOKEN_VALUE = os.environ.get("TABLEAU_TOKEN_VALUE")

In [ ]:
# connect to tableau server
tableau_auth = TSC.PersonalAccessTokenAuth(TABLEAU_TOKEN_NAME, TABLEAU_TOKEN_VALUE, TABLEAU_SITENAME)
server = TSC.Server(f"https://{TABLEAU_SERVER_URL}", use_server_version=True)
server.auth.sign_in(tableau_auth)

#### Data Sources - not used for now

In [ ]:
# get all datasources and their ID => ideally this is an ingestion job
# and the data sources are gonna be saved in a table
all_datasources, pagination_item = server.datasources.get()

# to make it easy, just save the data sources as dict, alongside their ID
datasources_dict = {i.name: i.id for i in all_datasources}

In [ ]:
# get the data source to refresh by ID
datasource = server.datasources.get_by_id(datasources_dict["Alarm RLS"])

# call the refresh method with the data source item
refreshed_datasource = server.datasources.refresh(datasource)

#### Workbooks

In [ ]:
# get all workbooks and their ID => ideally this is an ingestion job
# and the workbooks are gonna be saved in a table
# we also need projects
all_workbooks_items, pagination_item = server.workbooks.get()
all_project_items, pagination_item = server.projects.get()

# unpack payload in dict
#workbooks_dict = {i.name: i.id for i in all_workbooks_items}
workbooks = [(i.content_url, i.created_at, i.id, i.name, i.owner_id, i.project_id, i.project_name, i.size, i.show_tabs, i.hidden_views, i.tags, i.updated_at, i.webpage_url) for i in all_workbooks_items]


In [ ]:
all_workbooks_items[0]

In [ ]:
# these two dataframes are gonna be tables in snowflake
df_projects = pd.DataFrame([(i.id, i.parent_id, i.name) for i in all_project_items], columns=["project_id", "parent_project_id", "project_name"])

df_workbooks = pd.DataFrame(
    [(i.content_url, i.created_at, i.id, i.name, i.owner_id, i.project_id, i.project_name, i.size, i.show_tabs, i.hidden_views, i.tags, i.updated_at, i.webpage_url) for i in all_workbooks_items],
    columns=["content_url", "created_at", "id", "name", "owner_id", "project_id", "project_name", "size", "show_tabs", "hidden_views", "tags", "updated_at", "webpage_url"]
)

In [ ]:
df_workbooks.head()

##### This uses only Tableau metadata for triggering refreshes by specifying one or more projects of interest (can be done through a seed file)

In [ ]:
# here we apply some filters to get to the ID of the projects to refresh (basically Operation => Certified folder)
# this assumes a fixed structured; if this changes then we are not so flexible
proj_to_target = df_projects[df_projects["project_name"] == 'Operation'].project_id.values[0]
sub_proj_to_target = df_projects[(df_projects["parent_project_id"] == proj_to_target) & (df_projects["project_name"] == "3. Certified")].project_id.values[0]
project_to_refresh = df_projects[df_projects["parent_project_id"] == sub_proj_to_target].project_id.values

In [ ]:
project_to_refresh

In [ ]:
# here we get the workbook to refresh
workbooks_to_refresh = df_workbooks[df_workbooks["project_id"].isin(project_to_refresh)].id.values

In [ ]:
workbooks_to_refresh

In [ ]:
# loops through workbooks to refresh and trigger refresh
# run only if you want to refresh the specific dashboard
for w in workbooks_to_refresh:
    workbook = server.workbooks.get_by_id(w)
    # call the refresh method with the data source item
    refreshed_workbook = server.workbooks.refresh(workbook)
    print(f"refresh for workbook id {w} triggered")

##### This uses only Tableau metadata for triggering refreshes by specifying a specific tag (mantained through Tableau)

In [ ]:
# get all workbooks and their ID => ideally this is an ingestion job
# and the workbooks are gonna be saved in a table
# here if we use the tag we don't need the project info
all_workbooks_items, pagination_item = server.workbooks.get()

df_workbooks = pd.DataFrame(
    [(i.content_url, i.created_at, i.id, i.name, i.owner_id, i.project_id, i.project_name, i.size, i.show_tabs, i.hidden_views, i.tags, i.updated_at, i.webpage_url) for i in all_workbooks_items],
    columns=["content_url", "created_at", "id", "name", "owner_id", "project_id", "project_name", "size", "show_tabs", "hidden_views", "tags", "updated_at", "webpage_url"]
)

In [ ]:
df_workbooks.head()

In [ ]:
workbooks_to_refresh = df_workbooks[df_workbooks["tags"].astype(str).str.contains("daily")].id.values

In [ ]:
# we expect only one workbook to have a daily refresh
workbooks_to_refresh

In [ ]:
# loops through workbooks to refresh and trigger refresh
# run only if you want to refresh the specific dashboard
for w in workbooks_to_refresh:
    workbook = server.workbooks.get_by_id(w)
    # call the refresh method with the data source item
    refreshed_workbook = server.workbooks.refresh(workbook)
    print(f"refresh for workbook id {w} triggered")

In [ ]:
# close connections
server.auth.sign_out()

#### test package

In [4]:
from tableau_poc import TableauServer

In [5]:
tb_server = TableauServer(TABLEAU_TOKEN_NAME, TABLEAU_TOKEN_VALUE, TABLEAU_SITENAME, TABLEAU_SERVER_URL)

In [6]:
tb_server.get_workbooks()

[('Regional',
  datetime.datetime(2022, 10, 21, 7, 12, 4, tzinfo=<tableauserverclient.datetime_helpers.UTC object at 0x0000025D928DCE80>),
  '116426e6-11b4-4438-babd-1110d3df3554',
  'Regional',
  'af745434-6e79-4655-bb53-af3a3e427582',
  'fbf35beb-743d-4f44-b508-e461f6906d73',
  'Samples',
  2,
  True,
  None,
  set(),
  datetime.datetime(2022, 10, 21, 7, 12, 4, tzinfo=<tableauserverclient.datetime_helpers.UTC object at 0x0000025D928DCE80>),
  'https://dub01.online.tableau.com/#/site/factorypal/workbooks/1017403'),
 ('Superstore',
  datetime.datetime(2022, 10, 21, 7, 12, 9, tzinfo=<tableauserverclient.datetime_helpers.UTC object at 0x0000025D928DCE80>),
  '54135fc8-60ea-490d-aacf-52c0275c95d8',
  'Superstore',
  'af745434-6e79-4655-bb53-af3a3e427582',
  'fbf35beb-743d-4f44-b508-e461f6906d73',
  'Samples',
  1,
  True,
  None,
  set(),
  datetime.datetime(2022, 10, 21, 7, 12, 9, tzinfo=<tableauserverclient.datetime_helpers.UTC object at 0x0000025D928DCE80>),
  'https://dub01.online.tab